# LangGraph 101

[LLMs](https://python.langchain.com/docs/concepts/chat_models/) tornam possível incorporar inteligência em uma nova classe de aplicações. [LangGraph](https://langchain-ai.github.io/langgraph/) é um framework para ajudar a construir aplicações com LLMs. Aqui, vamos revisar o básico do LangGraph, explicar seus benefícios, mostrar como usá-lo para construir fluxos de trabalho / agentes, e mostrar como funciona com [LangChain](https://www.langchain.com/) / [LangSmith](https://docs.smith.langchain.com/).

![ecosystem](./img/ecosystem.png)

## Modelos de chat

[Modelos de chat](https://python.langchain.com/docs/concepts/chat_models/) são a base das aplicações LLM. Eles são tipicamente acessados através de uma interface de chat que recebe uma lista de [mensagens](https://python.langchain.com/docs/concepts/messages/) como entrada e retorna uma [mensagem](https://python.langchain.com/docs/concepts/messages/) como saída. LangChain fornece [uma interface padronizada para modelos de chat](https://python.langchain.com/api_reference/langchain/chat_models/langchain.chat_models.base.init_chat_model.html), facilitando [acessar muitos provedores diferentes](https://python.langchain.com/docs/integrations/chat/).

In [48]:
from dotenv import load_dotenv
load_dotenv()

True

In [50]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai", temperature=0)

## Executando o modelo

A interface `init_chat_model` fornece métodos [padronizados](https://python.langchain.com/docs/concepts/runnables/) para usar modelos de chat, que incluem:
- `invoke()`: Uma única entrada é transformada em uma saída.
- `stream()`: As saídas são [transmitidas](https://python.langchain.com/docs/concepts/streaming/#stream-and-astream) conforme são produzidas.

In [51]:
result = llm.invoke("O que é um agente?")

In [52]:
type(result)

langchain_core.messages.ai.AIMessage

In [53]:
from rich.markdown import Markdown
Markdown(result.content)

Um agente é, em sua essência, algo que percebe seu ambiente e atua nele para alcançar certos objetivos.            

Essa definição é bastante ampla e pode ser aplicada em diversos contextos, desde a biologia até a inteligência     
artificial.                                                                                                        

Vamos detalhar os componentes principais:                                                                          

 1 Percepção (Sensores): O agente precisa ser capaz de "sentir" ou "observar" o que está acontecendo em seu        
   ambiente. Isso pode ser feito através de:                                                                       
    • Sentidos biológicos: Olhos, ouvidos, tato, olfato (para humanos e animais).                                  
    • Sensores eletrônicos: Câmeras, microfones, radares, GPS, termômetros (para robôs e sistemas de IA).          
    • Dados de entrada: Informações digitais, comandos de usuário, dados de bancos de dados (para agentes de       
      software).                                                                                                   
 2 Ambiente: É o mundo ou o contexto onde o agente opera e interage. Pode ser:                                     
    • O mundo físico (para um robô, um carro autônomo).                                                            
    • Um sistema de computador, a internet (para um chatbot, um sistema de recomendação).                          
    • Um corpo humano (para um vírus ou bactéria).                                                                 
 3 Ação (Atuadores): O agente precisa ser capaz de "fazer" algo no ambiente, de influenciá-lo. Isso pode ser       
   através de:                                                                                                     
    • Movimento físico: Andar, pegar, empurrar (para humanos, animais, robôs).                                     
    • Comandos digitais: Enviar uma mensagem, exibir uma informação, modificar um arquivo, executar um programa    
      (para agentes de software).                                                                                  
    • Reações químicas/biológicas: Multiplicação, liberação de toxinas (para agentes biológicos).                  
 4 Objetivos: O agente geralmente tem uma razão para agir, um propósito. Pode ser:                                 
    • Sobreviver, reproduzir (para agentes biológicos).                                                            
    • Completar uma tarefa, otimizar um processo, responder a uma pergunta (para agentes de IA e software).        
    • Alcançar um estado desejado (para um sistema de controle).                                                   

Em resumo, um agente é uma entidade que:                                                                           

 • Observa (percebe) seu entorno.                                                                                  
 • Processa essa informação (toma decisões, raciocina).                                                            
 • Age (atua) no ambiente.                                                                                         
 • Com o objetivo de alcançar um fim específico.                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Exemplos de Agentes em diferentes contextos:                                                                       

 • Inteligência Artificial (IA):                                                                                   
    • Agente de software: Um chatbot que responde a perguntas, um sistema de recomendação que sugere produtos, um  
      web crawler que indexa páginas da internet.                                                                  
    • Agente robótico: Um carro autônomo que perc

## Ferramentas

[Ferramentas](https://python.langchain.com/docs/concepts/tools/) são utilitários que podem ser chamados por um modelo de chat. No LangChain, criar ferramentas pode ser feito usando o decorador `@tool`, que transforma funções Python em ferramentas chamáveis. Ele automaticamente inferirá o nome da ferramenta, descrição e argumentos esperados da definição da função. Você também pode usar [servidores Model Context Protocol (MCP)](https://github.com/langchain-ai/langchain-mcp-adapters) como ferramentas compatíveis com LangChain.

In [54]:
from langchain.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Escrever e enviar um email."""
    # Resposta placeholder - em aplicação real enviaria email
    return f"Email enviado para {to} com assunto '{subject}' e conteúdo: {content}"

In [55]:
type(write_email)

langchain_core.tools.structured.StructuredTool

In [56]:
write_email.args

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [57]:
Markdown(write_email.description)

Escrever e enviar um email.

## Chamada de Ferramentas

Ferramentas podem ser [chamadas](https://python.langchain.com/docs/concepts/tool_calling/) por LLMs. Quando uma ferramenta é vinculada ao modelo, o modelo pode escolher chamar a ferramenta retornando uma saída estruturada com argumentos da ferramenta. Usamos o método `bind_tools` para aumentar um LLM com ferramentas.

![tool-img](img/tool_call_detail.png)

Provedores frequentemente têm [parâmetros como `tool_choice`](https://python.langchain.com/docs/how_to/tool_choice/) para forçar a chamada de ferramentas específicas. `any` selecionará pelo menos uma das ferramentas.

Além disso, podemos [definir `parallel_tool_calls=False`](https://python.langchain.com/docs/how_to/tool_calling_parallel/) para garantir que o modelo só chamará uma ferramenta por vez (funcionalidade suportada apenas pela OpenAI e Anthropic).

In [58]:
# Conectar ferramentas a um modelo de chat com tool_choice="any" para forçar uso de ferramentas
model_with_tools = llm.bind_tools([write_email], tool_choice="any")

# O modelo agora será capaz de chamar ferramentas
output = model_with_tools.invoke("Rascunhe uma resposta para meu chefe (chefe@empresa.com.br) sobre a reunião de amanhã")

In [59]:
type(output)

langchain_core.messages.ai.AIMessage

In [60]:
output

AIMessage(content='', additional_kwargs={'function_call': {'name': 'write_email', 'arguments': '{"to": "chefe@empresa.com.br", "content": "Prezado(a) Chefe, confirmo minha presen\\u00e7a na reuni\\u00e3o de amanh\\u00e3. Atenciosamente, [Seu Nome]", "subject": "Reuni\\u00e3o de amanh\\u00e3"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f3be2f3f-34cc-4b2f-9d39-a66f45634793-0', tool_calls=[{'name': 'write_email', 'args': {'to': 'chefe@empresa.com.br', 'content': 'Prezado(a) Chefe, confirmo minha presença na reunião de amanhã. Atenciosamente, [Seu Nome]', 'subject': 'Reunião de amanhã'}, 'id': '17c695e9-ee34-4b76-b473-70df21f17415', 'type': 'tool_call'}], usage_metadata={'input_tokens': 89, 'output_tokens': 65, 'total_tokens': 238, 'input_token_details': {'cache_read': 0}})

In [61]:
# Extrair chamadas de ferramenta e executá-las
if output.tool_calls:
	args = output.tool_calls[0]['args']
	args
else:
	print("Nenhuma chamada de ferramenta encontrada na resposta do modelo.")
	args = None
	args

In [ ]:
# Extrair os argumentos da chamada de ferramenta do output
if hasattr(output, "tool_calls") and output.tool_calls:
	args = output.tool_calls[0]['args']
	result = write_email.invoke(args)
	print(result)
else:
	print("Nenhuma chamada de ferramenta encontrada na resposta do modelo.")

Email enviado para chefe@empresa.com.br com assunto 'Reunião de amanhã' e conteúdo: Prezado(a) Chefe, confirmo minha presença na reunião de amanhã. Atenciosamente, [Seu Nome]


![basic_prompt](img/tool_call.png)

## Fluxos de Trabalho
 
Existem muitos padrões para construir aplicações com LLMs.

[Podemos incorporar chamadas de LLM em fluxos de trabalho pré-definidos](https://langchain-ai.github.io/langgraph/tutorials/workflows/), dando ao sistema mais autonomia para tomar decisões.

Como exemplo, poderíamos adicionar uma etapa de roteador para determinar se devemos escrever um email ou não.

![workflow_example](img/workflow_example.png)

## Agentes

Podemos aumentar ainda mais a autonomia, permitindo que o LLM direcione dinamicamente seu próprio uso de ferramentas.

[Agentes](https://langchain-ai.github.io/langgraph/tutorials/workflows/#agent) são tipicamente implementados como chamada de ferramenta em um loop, onde a saída de cada chamada de ferramenta é usada para informar a próxima ação.

![agent_example](img/agent_example.png)

Agentes são bem adequados para problemas abertos onde é difícil prever os passos *exatos* necessários com antecedência.

Fluxos de trabalho são frequentemente apropriados quando o fluxo de controle pode ser facilmente definido com antecedência.

![workflow_v_agent](img/workflow_v_agent.png)

## O que é LangGraph?

[LangGraph](https://langchain-ai.github.io/langgraph/concepts/high_level/) fornece infraestrutura de suporte de baixo nível que fica por baixo de *qualquer* fluxo de trabalho ou agente.

Ele não abstrai prompts ou arquitetura, e fornece alguns benefícios:

- **Controle**: Facilita definir e/ou combinar agentes e fluxos de trabalho.
- **Persistência**: Fornece uma maneira de persistir o estado de um grafo, o que permite tanto memória quanto humano-no-loop.
- **Teste, Depuração e Deployment**: Fornece uma rampa fácil para testar, depurar e fazer deploy de aplicações.

### Controle

LangGraph permite definir sua aplicação como um grafo com:

1. *Estado*: Que informações precisamos rastrear ao longo da aplicação?
2. *Nós*: Como queremos atualizar essas informações ao longo da aplicação?
3. *Arestas*: Como queremos conectar esses nós?

Podemos usar a classe [`StateGraph`](https://langchain-ai.github.io/langgraph/concepts/low_level/#graphs) para inicializar um grafo LangGraph com um objeto [`State`](https://langchain-ai.github.io/langgraph/concepts/low_level/#state).

`State` define o esquema para as informações que queremos rastrear ao longo da aplicação.

Isso pode ser qualquer objeto com `getattr()` em python, como um dicionário, dataclass, ou objeto Pydantic:

- TypeDict é mais rápido mas não suporta padrões
- Dataclass é basicamente tão rápido, suporta sintaxe de ponto `state.foo`, e tem padrões.
- Pydantic é mais lento (especialmente com validadores customizados) mas fornece validação de tipo.

In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END, MessagesState

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)

Cada nó é simplesmente uma função python ou código typescript. Isso nos dá controle total sobre a lógica dentro de cada nó.

Eles recebem o estado atual, e retornam um dicionário para atualizar o estado.

Por padrão, [chaves de estado são sobrescritas](https://langchain-ai.github.io/langgraph/how-tos/state-reducers/).

No entanto, você pode [definir lógica de atualização customizada](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers).

![nodes_edges](img/nodes_edges.png)

In [ ]:
def write_email_node(state: StateSchema) -> StateSchema:
    # Código imperativo que processa a solicitação
    output = model_with_tools.invoke(state["request"])

    # Verificar se o modelo fez chamadas de ferramenta
    if hasattr(output, "tool_calls") and output.tool_calls:
        args = output.tool_calls[0]['args']
        email = write_email.invoke(args)
        return {"email": email}
    else:
        # Se não houver chamadas de ferramenta, retornar resposta como texto
        return {"email": f"Resposta do modelo: {output.content}"}

Arestas conectam nós.

Especificamos o fluxo de controle adicionando arestas e nós ao nosso grafo de estado.

In [28]:
workflow = StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

In [29]:
app.invoke({"request": "Rascunhe uma resposta para meu chefe (chefe@empresa.com.br) sobre a reunião de amanhã"})

{'request': 'Rascunhe uma resposta para meu chefe (chefe@empresa.com.br) sobre a reunião de amanhã',
 'email': 'Resposta do modelo: Ok, qual seria o assunto e o conteúdo do e-mail?'}

O roteamento entre nós pode ser feito [condicionalmente](https://langchain-ai.github.io/langgraph/concepts/low_level/#conditional-edges) usando uma função simples.

O valor de retorno desta função é usado como o nome do nó (ou lista de nós) para o qual enviar o estado a seguir.

Você pode opcionalmente fornecer um dicionário que mapeia a saída `should_continue` para o nome do próximo nó.

In [ ]:
from email_assistant.utils import show_graph

def call_llm(state: MessagesState) -> MessagesState:
    """Executar LLM"""

    output = model_with_tools.invoke(state["messages"])
    return {"messages": [output]}

def run_tool(state: MessagesState):
    """Executa a chamada de ferramenta"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        observation = write_email.invoke(tool_call["args"])
        result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})
    return {"messages": result}

def should_continue(state: MessagesState) -> Literal["run_tool", "__end__"]:
    """Rotear para manipulador de ferramenta, ou terminar se ferramenta Done foi chamada"""

    # Obter a última mensagem
    messages = state["messages"]
    last_message = messages[-1]

    # Se a última mensagem for uma chamada de ferramenta, verificar se é uma chamada de ferramenta Done
    if last_message.tool_calls:
        return "run_tool"
    # Caso contrário, paramos (respondemos ao usuário)
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("run_tool", run_tool)
workflow.add_edge(START, "call_llm")
workflow.add_conditional_edges("call_llm", should_continue, {"run_tool": "run_tool", END: END})
workflow.add_edge("run_tool", END)

# Execute o fluxo de trabalho
app = workflow.compile()

In [ ]:
show_graph(app)

In [ ]:
result = app.invoke({"messages": [{"role": "user", "content": "Rascunhe uma resposta para meu chefe (chefe@empresa.com.br) confirmando que quero participar do Interrupt!"}]})
for m in result["messages"]:
    m.pretty_print()

Com esses componentes de baixo nível, você pode construir muitos fluxos de trabalho e agentes diferentes. Veja [este tutorial](https://langchain-ai.github.io/langgraph/tutorials/workflows/)!

Como agentes são um padrão tão comum, o [LangGraph](https://langchain-ai.github.io/langgraph/tutorials/workflows/#pre-built) tem [uma abstração de agente pré-construída](https://langchain-ai.github.io/langgraph/agents/overview/?ref=blog.langchain.dev#what-is-an-agent).

Com o [método pré-construído](https://langchain-ai.github.io/langgraph/tutorials/workflows/#pre-built) do LangGraph, apenas passamos o LLM, ferramentas e prompt.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Responda à solicitação do usuário usando as ferramentas fornecidas."
)

# Execute o agente
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Rascunhe uma resposta para meu chefe (chefe@empresa.com.br) confirmando que quero participar do Interrupt!"}]}
)

for m in result["messages"]:
    m.pretty_print()

### Persistência

#### Threads

Pode ser muito útil permitir que agentes pausem durante tarefas de longa duração.

LangGraph tem uma camada de persistência integrada, implementada através de checkpointers, para permitir isso.

Quando você compila o grafo com um checkpointer, o checkpointer salva um [checkpoint](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpoints) do estado do grafo a cada passo.

Checkpoints são salvos em uma thread, que pode ser acessada após a execução do grafo ser completada.

![checkpointer](img/checkpoints.png)

Compilamos o grafo com um [checkpointer](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-libraries).

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Responda à solicitação do usuário usando as ferramentas fornecidas.",
    checkpointer=InMemorySaver()
)

config = {"configurable": {"thread_id": "1"}}
result = agent.invoke({"messages": [{"role": "user", "content": "Quais são algumas boas práticas para escrever emails?"}]}, config)

In [ ]:
# Obter o snapshot de estado mais recente
config = {"configurable": {"thread_id": "1"}}
state = agent.get_state(config)
for message in state.values['messages']:
    message.pretty_print()

In [ ]:
# Continue a conversa
result = agent.invoke({"messages": [{"role": "user", "content": "Bom, vamos usar a lição 3 para rascunhar uma resposta para meu chefe confirmando que quero participar do Interrupt"}]}, config)
for m in result['messages']:
    m.pretty_print()

In [ ]:
# Continue a conversa
result = agent.invoke({"messages": [{"role": "user", "content": "Gostei disso, vamos escrever o email para chefe@empresa.com.br"}]}, config)
for m in result['messages']:
    m.pretty_print()

#### Interrupções

No LangGraph, também podemos usar [interrupções](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/wait-user-input/) para parar a execução do grafo em pontos específicos.

Frequentemente isso é usado para coletar entrada de um usuário e continuar a execução com a entrada coletada.

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver
from email_assistant.utils import show_graph

class State(TypedDict):
    input: str
    user_feedback: str

def step_1(state):
    print("---Step 1---")
    pass

def human_feedback(state):
    print("---human_feedback---")
    feedback = interrupt("Por favor, forneça feedback:")
    return {"user_feedback": feedback}

def step_3(state):
    print("---Step 3---")
    pass

builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Configurar memória
memory = InMemorySaver()

# Adicionar
graph = builder.compile(checkpointer=memory)

In [ ]:
show_graph(graph)

In [ ]:
# Entrada
initial_input = {"input": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Execute o grafo até a primeira interrupção
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

Para retomar de uma interrupção, podemos usar [o objeto `Command`](https://langchain-ai.github.io/langgraph/how-tos/command/). 

Vamos usá-lo para retomar o grafo do estado interrompido, passando o valor para retornar da chamada de interrupção para `resume`.

In [ ]:
# Continue a execução do grafo
for event in graph.stream(
    Command(resume="ir para o passo 3!"),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

### Rastreamento

Quando estamos usando LangChain ou LangGraph, o logging do LangSmith [funcionará automaticamente](https://docs.smith.langchain.com/observability/how_to_guides/trace_with_langgraph) com as seguintes variáveis de ambiente definidas:

```
export LANGSMITH_TRACING=true
export LANGSMITH_API_KEY="<sua-chave-api-langsmith>"
```

Aqui está o rastreamento LangSmith da execução do agente acima:

https://smith.langchain.com/public/6f77014f-d054-44ed-aa2c-8b06ceab689f/r

Podemos ver que o agente é capaz de continuar a conversa do estado anterior porque usamos um checkpointer.

Também podemos implantar nosso grafo usando a [LangGraph Platform](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/). 

Isso cria um servidor [com uma API](https://langchain-ai.github.io/langgraph/cloud/reference/api/api_ref.html) que podemos usar para interagir com nosso grafo e um IDE interativo, o LangGraph [Studio](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/).

Precisamos apenas garantir que nosso projeto tenha [uma estrutura](https://langchain-ai.github.io/langgraph/concepts/application_structure/) como esta:

Aqui podemos ver uma visualização do grafo bem como o estado do grafo no Studio.

![langgraph_studio](img/langgraph_studio.png)

Além disso, você pode ver a documentação da API para a implantação local aqui:

http://127.0.0.1:2024/docs